PREPARING THE ENVIRONMENT

In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
sudo apt-get install -f

INSTALLAZIONE MODULI E PACCHETTI

In [ ]:
!pip install chromedriver-py
!pip install selenium

In [ ]:
# IMPORT LIBRARIES
import sys
import logging
from selenium.webdriver.remote.remote_connection import LOGGER
LOGGER.setLevel(logging.WARNING)
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from tqdm.notebook import tqdm
import pandas as pd
import json
import pprint
from chromedriver_py import binary_path
import time
import random

SETTING UP THE BROWSER

In [ ]:
chrome_options = webdriver.ChromeOptions()  # Initialize Chrome options
chrome_options.add_argument('--headless')  # Run Chrome in headless mode - In our local notebook we can remove the `--headless` option.
chrome_options.add_argument('--no-sandbox')  # Disable Chrome’s default sandboxing behavior
chrome_options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
chrome_options.add_argument("window-size=1900,800")  # Set the window size for the browser
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")  # Set a custom user agent

In [ ]:
service = Service(executable_path=binary_path)  # Create a Service object with the path to the ChromeDriver executable
wd = webdriver.Chrome(service=service, options=chrome_options)  # Initialize the WebDriver with the specified service and options

CONNESSIONE AL SITO

In [ ]:
wd.get("https://www.facebook.com/login/?locale=it_IT")

# Wait for N seconds to allow the page to fully load
N = 5
time.sleep(N)


In [ ]:
wd.save_screenshot('screenshot.png')

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img = mpimg.imread('/content/screenshot.png')
plt.figure(figsize=(20, 10))
imgplot = plt.imshow(img)
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
email = wd.find_elements(By.CSS_SELECTOR, "input.inputtext._55r1.inputtext._1kbt.inputtext._1kbt")[0]
email.send_keys("f.bogazzi@campus.unimib.it")

In [ ]:
password = wd.find_elements(By.CSS_SELECTOR, "input.inputtext._55r1.inputtext._9npi.inputtext._9npi")[0]
password.send_keys("##############")

In [ ]:
from selenium.webdriver.common.keys import Keys
email.send_keys(Keys.RETURN)

In [ ]:
search = wd.find_elements(By.CSS_SELECTOR, "input.x1i10hfl.xggy1nq.x1s07b3s.x1kdt53j.x1yc453h.xhb22t3.xb5gni.xcj1dhv.x2s2ed0.xq33zhf.xjyslct.xjbqb8w.xnwf7zb.x40j3uw.x1s7lred.x15gyhx8.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.xzsf02u.xdl72j9.x1iyjqo2.xs83m0k.xjb2p0i.x6prxxf.xeuugli.x1a2a7pz.x1n2onr6.x15h3p50.xm7lytj.x1sxyh0.xdvlbce.xurb0ha.x1vqgdyp.x1xtgk1k.x17hph69.xo6swyp.x1ad04t7.x1glnyev.x1ix68h3.x19gujb8")[0]
search.send_keys("#siccità")

In [ ]:
search.send_keys(Keys.RETURN)

INIZIO SCRAPING

In [ ]:
#imposta se continuare da dove si è fermato. se true ricomincia
start_scrape = True
if start_scrape == True:
  columns = ['count','testo'] #'interazioni']
  #Crea un DataFrame vuoto con le colonne definite
  df_posts = pd.DataFrame(columns=columns)
  wd.execute_script("window.scrollTo(0, 0);")

In [ ]:
# SCRAPING
post_reel = wd.find_elements(By.CSS_SELECTOR, "div.x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z")

post_reel_saved = set()
insert_has_done = True
num_attempt = 0
count = 0

while insert_has_done == True or num_attempt == 1:
  insert_has_done = False
  for post in post_reel:
    if post.id not in post_reel_saved:
      try:
        count += 1
        testo = post.find_element(By.CSS_SELECTOR, 'div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x1vvkbs.x126k92a').text
        #interazioni = post.find_element(By.CSS_SELECTOR, 'div.x1ey2m1c.x78zum5.xds687c.xjkvuk6.x10l6tqk.xni59qk').text
        row = [count, testo] #interazioni]
        df_posts.loc[len(df_posts)] = row
        print(len(df_posts))
        insert_has_done = True
        num_attempt = 0
        post_reel_saved.add(post.id)
      except NoSuchElementException:
        pass
        #print('Pubblicità')
        #print('--------------')
  if insert_has_done == False:
    num_attempt = num_attempt + 1

  wd.execute_script("window.scrollBy(0, 1000);")
  post_reel = wd.find_elements(By.CSS_SELECTOR, "div.x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z")
  random_number = random.uniform(1.5, 3.5)
  time.sleep(random_number)



In [ ]:
df_posts